In [1]:
import pandas as pd
import numpy as np
import os
import json
import joblib

import altair as alt
import vegafusion as vf
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
# Pre-define chart function
def chart(df, x, y, title, color=alt.value('steelblue'), width=480, height=320):
    return alt.Chart(df).encode(
        x=x,
        y=y,
        color=color,
    ).properties(
    title=title,
    width=width,
    height=height,
).configure(
    axis=alt.AxisConfig(
        domain=False, # remove axis line
        ticks=False, # remove ticks
        labelAngle=0, # rotate labels
        labelColor='gray', # color of labels
        labelFontSize=10,
    ),
    font='Helvetica Neue',
    view=alt.ViewConfig(stroke=None), # remove border
)

## 1 - Load the training data

In [3]:
input_path = os.path.join('..', 'data', 'cleaned')

train_df = pd.read_parquet(os.path.join(input_path, 'train.parquet'))
train_df.head()

,file,VMONTH,VYEAR,VDAYR,AGE,AGER,SEX,ETHNIC,RACE,USETOBAC,...,PHYS,PHYSASST,NPNMW,RNLPN,OTHPROV,MHP,REFOTHMD,RETAPPT,OTHDISP,ERADMHOS
0,opd2006,12,2006,6,55,4,2,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
1,opd2006,11,2006,5,66,5,2,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
2,opd2006,11,2006,4,71,5,1,2,1,1,...,1,0,0,0,0,<NA>,1,0,0,0
3,opd2006,11,2006,3,1,1,1,2,1,1,...,1,0,0,0,0,<NA>,0,0,0,0
4,opd2006,11,2006,2,21,2,1,2,1,2,...,1,0,0,0,0,<NA>,0,0,0,0


## 2 - Load the vairiables dictionary and define features for clustering

In [4]:
# Load the variables dictionary
with open(os.path.join(input_path, 'variables.json'), 'r') as f:
    variables = json.load(f)

print(f'Variable Categories:\n')
for category, list in variables.items():
    print(f'{category}')
    print(f'{list}')

Variable Categories:

dateOfVisit
['VMONTH', 'VYEAR', 'VDAYR']
demographics
['AGE', 'AGER', 'SEX', 'ETHNIC', 'RACE', 'USETOBAC']
payment
['PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER']
visitReason
['INJDET', 'MAJOR', 'RFV1', 'RFV2', 'RFV3']
patientClinicHistory
['SENBEFOR', 'PASTVIS']
vitalSigns
['HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS']
imputedFields
['BDATEFL', 'SEXFL', 'SENBEFL', 'PASTFL']
physicianDiagnoses
['DIAG1', 'DIAG2', 'DIAG3']
differentialDiagnoses
['PRDIAG1', 'PRDIAG2', 'PRDIAG3']
presentSymptomsStatus
['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
services
['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 'CHLAMYD', 'PAPCONV', 'PAPLIQ', 'PAPUNSP', 'HPVDNA', 'EKG', 'URINE', 'HTTAKE', 'WTTAKE


### 2.1 Defining features for clustering

##### !!! The statistical test result of the features should be referred first

In [5]:
train_df[variables['visitReason']].value_counts()

INJDET  MAJOR  RFV1   RFV2   RFV3 
5       2      48000  44100  41150    40
1       2      23200  48000  44100    25
5       1      10100  14400  14000    21
               14400  14000  14551    21
1       3      46050  47350  11500    20
                                      ..
5       1      14400  15950  15300     1
                             14750     1
                             10200     1
                      15702  41150     1
        5      89980  45650  42050     1
Name: count, Length: 7796, dtype: int64

In [6]:
# Defining the independent variables as features for classification
features = \
    ['AGE', 'SEX', 'USETOBAC'] + variables['visitReason'] + ['PASTVIS'] + variables['vitalSigns'] \
    + variables['presentSymptomsStatus']

print(f'Features: {features}')
print(f'Number of Features: {len(features)}')

Features: ['AGE', 'SEX', 'USETOBAC', 'INJDET', 'MAJOR', 'RFV1', 'RFV2', 'RFV3', 'PASTVIS', 'HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
Number of Features: 30


In [7]:
X_train = train_df.loc[:, features]

## 3 - Preprocess and engineer the features

### 3.1 - Binning of quantitative variables to categorical features
Bin the following quantitative variables:

AGE, RFV1, RFV2, RFV3, BMI, TEMPF, BPSYS, BPDIAS

#### 3.1.1 - Bin the AGE variable
Do we bin as recoded age groups (`AGER`) or as each 20 years?

In [8]:
# Check the distribution of `AGER`
# 1 = Under 15 years 
# 2 = 15-24 years 
# 3 = 25-44 years 
# 4 = 45-64 years 
# 5 = 65-74 years 
# 6 = 75 years and over|
 
chart(
    df=train_df,
    x='AGER:O',
    y='count()',
    title='Distribution of AGER',
).mark_bar().interactive()

alt.Chart(...)

In [9]:
# Bin the AGE variable as age groups
# 0-2 = Infant
# 2-4 = Toddler
# 4-12 = Child
# 12-20 = Teenager
# 20-40 = Adult
# 40-60 = Middle Aged
# >= 60 = Senior

age_groups = ['Infant', 'Toddler', 'Child', 'Teenager', 'Child or Teenager', 'Adult', 'Middle Aged', 'Senior']

def bin_age(age):
    #if age < 2: return 'Infant'
    #elif age < 4: return 'Toddler'
    #elif age < 12: return 'Child'
    #elif age < 20: return 'Teenager'
    if age < 20: return 'Child or Teenager'
    elif age < 40: return 'Adult'
    elif age < 60: return 'Middle Aged'
    else: return 'Senior'
    

X_train['AGE_GROUP'] = X_train['AGE'].apply(bin_age)

# Check the distribution of age groups
chart(
    df=X_train,
    x=alt.X('AGE_GROUP:O', sort=age_groups),
    y='count()',
    title='Distribution of AGE GROUPS',
).mark_bar().interactive()

alt.Chart(...)

#### 3.1.2 - Bin the Reason For Visit variables
RFV1, RFV2, RFV3

In [10]:
# Load the REASON FOR VISIT classification summary of codes
rvf_summary = pd.read_excel(os.path.join('..', 'data', 'raw', 'RFV_codes_summary.xlsx'))

# Split the 'CODE NUMBER' column into 'START' and 'END' columns
rvf_summary[['START', 'END']] = rvf_summary['CODE NUMBER'].str.split('-', expand=True).astype(int)

rvf_summary.sample(5)

,MODULE_1,MODULE_2,CODE NUMBER,START,END
24,DISEASE MODULE,Congenital Anomalies,2950-2979,2950,2979
13,DISEASE MODULE,Diseases of the Blood and Blood-forming Organs,2250-2299,2250,2299
2,SYMPTOM MODULE,Symptoms Referable to the Nervous System (Excl...,1200-1259,1200,1259
33,TREATMENT MODULE,Specific Types of Therapy,4400-4499,4400,4499
35,TREATMENT MODULE,Medical Counseling,4600-4699,4600,4699


In [11]:
# Find the `START` and `END` range, 
# and map the corresponding `MODULE_2` to X_train,
# according to the value of `RFV1`

def get_module(code):
    return rvf_summary[(rvf_summary['START'] <= code) & (rvf_summary['END'] >= code)]['MODULE_2'].values[0]


X_train['RFV1_MOD'] = X_train['RFV1'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV1_MOD unique values: {X_train["RFV1_MOD"].value_counts()}')

X_train['RFV2_MOD'] = X_train['RFV2'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV2_MOD unique values: {X_train["RFV2_MOD"].value_counts()}')

X_train['RFV3_MOD'] = X_train['RFV3'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
print(f'RFV3_MOD unique values: {X_train["RFV3_MOD"].value_counts()}')

RFV1_MOD unique values: RFV1_MOD
Progress Visit, NEC                                                  9641
Symptoms Referable to the Musculoskeletal System                     8232
Special Examinations                                                 7416
General Examinations                                                 6895
Symptoms Referable to the Respiratory System                         5908
General Symptoms                                                     4536
Injury by Type and/or Location                                       4312
Symptoms Referable to Psychological and Mental Disorders             4048
Symptoms Referable to the Genitourinary System                       3847
Diagnostic Tests                                                     3801
Symptoms Referable to the Digestive System                           3301
Preoperative and Postoperative Care                                  2971
Symptoms Referable to the Skin, Nails, and Hair                      2845
Sympt

#### 3.1.3 - Bin the vitalSigns variables
BMI, TEMPF, BPSYS, BPDIAS

In [12]:
# Bin the BMI as weight status
# <18.5 = Underweight
# 18.5-25 = Normal weight
# 25-30 = Overweight
# >=30 = Obesity

bmi_groups = ['Underweight', 'Normal weight', 'Overweight', 'Obesity']

def bin_bmi(bmi):
    if bmi < 18.5: return 'Underweight'
    elif bmi < 25: return 'Normal weight'
    elif bmi < 30: return 'Overweight'
    else: return 'Obesity'

X_train['BMI_GROUP'] = X_train['BMI'].apply(bin_bmi)

# Check the distribution of BMI groups
chart(
    df=X_train,
    x=alt.X('BMI_GROUP:O', sort=bmi_groups),
    y='count()',
    title='Distribution of BMI GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [13]:
# Bin the TEMPF as fever status
# <95 = Hypothermia
# 95-99 = Normal temperature
# 99-100 = Low grade fever
# 100-103 = Fever
# >=103 = Hyperpyrexia

tempf_groups = ['Hypothermia', 'Normal temperature', 'Low grade fever', 'Fever', 'Hyperpyrexia']

def bin_tempf(tempf):
    if tempf < 95: return 'Hypothermia'
    elif tempf < 99: return 'Normal temperature'
    #elif tempf < 100: return 'Low grade fever'
    elif tempf < 103: return 'Fever'
    else: return 'Hyperpyrexia'

X_train['TEMPF_GROUP'] = X_train['TEMPF'].apply(bin_tempf)

# Check the distribution of TEMPF groups
chart(
    df=X_train,
    x=alt.X('TEMPF_GROUP:O', sort=tempf_groups),
    y='count()',
    title='Distribution of TEMPF GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [14]:
# Bin the BPSYS as systolic blood pressure status
# <90 = Hypotension
# 90-120 = Normal blood pressure
# 120-140 = Prehypertension
# >=140 = Hypertension

bpsys_groups = ['Hypotension', 'Normal blood pressure', 'Prehypertension', 'Hypertension']

def bin_bpsys(bpsys):
    if bpsys < 90: return 'Hypotension'
    elif bpsys < 120: return 'Normal blood pressure'
    elif bpsys < 140: return 'Prehypertension'
    else: return 'Hypertension'

X_train['BPSYS_GROUP'] = X_train['BPSYS'].apply(bin_bpsys)

# Check the distribution of BPSYS groups
chart(
    df=X_train,
    x=alt.X('BPSYS_GROUP:O', sort=bpsys_groups),
    y='count()',
    title='Distribution of BPSYS GROUPS',
).mark_bar().interactive()

alt.Chart(...)

In [15]:
# Bin the BPDIAS as diastolic blood pressure status
# <60 = Low diastolic blood pressure
# 60-90 = Normal diastolic blood pressure
# 90-110 = High diastolic blood pressure
# >=110 = Hypertension

bpdias_groups = [
    'Low diastolic blood pressure', 'Normal diastolic blood pressure', 'High diastolic blood pressure', 'Hypertension'
]

def bin_bpdias(bpdias):
    if bpdias < 60: return 'Low diastolic blood pressure'
    elif bpdias < 90: return 'Normal diastolic blood pressure'
    elif bpdias < 110: return 'High diastolic blood pressure'
    else: return 'Hypertension'

X_train['BPDIAS_GROUP'] = X_train['BPDIAS'].apply(bin_bpdias)

# Check the distribution of BPDIAS groups
chart(
    df=X_train,
    x=alt.X('BPDIAS_GROUP:O', sort=bpdias_groups),
    y='count()',
    title='Distribution of BPDIAS GROUPS',
).mark_bar().interactive()

alt.Chart(...)

#### 3.1.4 - Define the list of bin features for the following quantitative variables:

AGE, RFV1, RFV2, RFV3, BMI, TEMPF, BPSYS, BPDIAS

In [16]:
bin_features = ['AGE_GROUP', 'RFV1_MOD', 'RFV2_MOD', 'RFV3_MOD', 'BMI_GROUP', 'TEMPF_GROUP', 'BPSYS_GROUP', 'BPDIAS_GROUP']

### 3.2 - Normalization of the rest quantitative variables
Normalize the following quantitative variables:

PASTVIS, HTIN, WTLB

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
quantitative_features = ['PASTVIS', 'HTIN', 'WTLB']

# Normalize quantitative features
scaler = StandardScaler()
X_train[quantitative_features] = scaler.fit_transform(X_train[quantitative_features])

# Check the result
X_train[quantitative_features].head()

,PASTVIS,HTIN,WTLB
0,-0.133476,0.530558,1.762024
1,-0.554639,0.609827,0.586196
2,0.147299,-0.103596,0.329405
3,-0.554639,-1.926787,-1.670855
4,0.147299,0.372019,-0.035507


### 3.3 - Create interaction features
AGE, HTIN, WTLB, BMI, BPSYS, BPDIAS, CEBVD, CHF, DIABETES, HYPLIPID, HTN, OBESITY

### 3.4 - Encode categorical features

#### 3.4.1 - Encode binary categorical features

In [19]:
# Check binary categorical features
binary_features = [feature for feature in features if X_train[feature].nunique() == 2]
print(f'Binary Features: {binary_features}')
print(f'Number of Binary Features: {len(binary_features)}')

# Encode binary features as True or False and skip NaN values
X_train[binary_features] = X_train[binary_features].map(lambda x: True if x == 1 else False)

# Check the result
X_train[binary_features].head()

Binary Features: ['SEX', 'USETOBAC', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
Number of Binary Features: 16


,SEX,USETOBAC,ARTHRTIS,ASTHMA,CANCER,CEBVD,CHF,CRF,COPD,DEPRN,DIABETES,HYPLIPID,HTN,IHD,OBESITY,OSTPRSIS
0,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,False
1,False,True,False,False,False,False,False,False,True,False,False,True,True,False,False,False
2,True,True,False,False,True,False,False,False,False,False,True,True,True,False,True,False
3,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


#### 3.4.2 - Encode non-binary categorical features

In [20]:
# Check non-binary categorical features
non_binary_features = ['INJDET', 'MAJOR', 'CASTAGE'] + bin_features

print(f'Non-binary Categorical Features: {non_binary_features}')
print(f'Number of Non-binary Categorical Features: {len(non_binary_features)}')

Non-binary Categorical Features: ['INJDET', 'MAJOR', 'CASTAGE', 'AGE_GROUP', 'RFV1_MOD', 'RFV2_MOD', 'RFV3_MOD', 'BMI_GROUP', 'TEMPF_GROUP', 'BPSYS_GROUP', 'BPDIAS_GROUP']
Number of Non-binary Categorical Features: 11


In [21]:
# Encode non-binary categorical features
X_train = pd.get_dummies(X_train, columns=non_binary_features)

# Get the list of encoded non-binary features
encoded_non_binary_features = [feature for feature in X_train.columns if feature not in features + binary_features]

print(f'Encoded Non-binary Categorical Features: {encoded_non_binary_features}')
print(f'Number of Encoded Non-binary Categorical Features: {len(encoded_non_binary_features)}')

Encoded Non-binary Categorical Features: ['INJDET_1', 'INJDET_2', 'INJDET_3', 'INJDET_4', 'INJDET_5', 'MAJOR_1', 'MAJOR_2', 'MAJOR_3', 'MAJOR_4', 'MAJOR_5', 'CASTAGE_0', 'CASTAGE_1', 'CASTAGE_2', 'CASTAGE_3', 'AGE_GROUP_Adult', 'AGE_GROUP_Child or Teenager', 'AGE_GROUP_Middle Aged', 'AGE_GROUP_Senior', 'RFV1_MOD_ADMINISTRATIVE MODULE', 'RFV1_MOD_Congenital Anomalies', 'RFV1_MOD_Diagnostic Tests', 'RFV1_MOD_Diseases of the Blood and Blood-forming Organs', 'RFV1_MOD_Diseases of the Circulatory System', 'RFV1_MOD_Diseases of the Digestive System', 'RFV1_MOD_Diseases of the Ear', 'RFV1_MOD_Diseases of the Eye', 'RFV1_MOD_Diseases of the Genitourinary System', 'RFV1_MOD_Diseases of the Musculoskeletal System and Connective Tissue', 'RFV1_MOD_Diseases of the Nervous System', 'RFV1_MOD_Diseases of the Respiratory System', 'RFV1_MOD_Diseases of the Skin and Subcutaneous Tissue', 'RFV1_MOD_Endocrine, Nutritional, Metabolic, and Immunity Diseases', 'RFV1_MOD_Family Planning', 'RFV1_MOD_General E

### 3.5 - Dimensionality reduction for categorical features

### 3.6 - Redefine the classification DataFrame for training

In [22]:
# Redefine the classification DataFrame
X_train = X_train.loc[:, quantitative_features + binary_features + encoded_non_binary_features]

# Check the shape of the classification DataFrame
print(f'Classification DataFrame Shape: {X_train.shape}')

Classification DataFrame Shape: (103486, 185)


### 3.7 - Check missing values in X_train

In [23]:
# Check the missing values in X_train
X_train.isnull().sum()

PASTVIS                                         19878
HTIN                                            56172
WTLB                                            34554
SEX                                                 0
USETOBAC                                            0
                                                ...  
BPSYS_GROUP_Prehypertension                         0
BPDIAS_GROUP_High diastolic blood pressure          0
BPDIAS_GROUP_Hypertension                           0
BPDIAS_GROUP_Low diastolic blood pressure           0
BPDIAS_GROUP_Normal diastolic blood pressure        0
Length: 185, dtype: int64

## 4 - Prepare dependent variables

In [24]:
# Check the missing values in 'DIAG1', 'DIAG2', and 'DIAG3'
print(f'Missing Values in DIAG1: {train_df["DIAG1"].isna().sum()}')
print(f'Missing Values in DIAG2: {train_df["DIAG2"].isna().sum()}')
print(f'Missing Values in DIAG3: {train_df["DIAG3"].isna().sum()}')
print()

# Check the numbers of ruled out or questionable diagnoses
# (when 'PRDIAG1', 'PRDIAG2', and 'PRDIAG3' equals to 1)
print(f'Number of Ruled Out Diagnoses in DIAG1: {train_df["PRDIAG1"].sum()}')
print(f'Number of Ruled Out Diagnoses in DIAG2: {train_df["PRDIAG2"].sum()}')
print(f'Number of Ruled Out Diagnoses in DIAG3: {train_df["PRDIAG3"].sum()}')
print()

# Check the number of samples with missing 'DIAG1' and 'PRDIAG1' equals to 1
print(f'Number of Samples with Missing DIAG1 and PRDIAG1 equals to 1: {train_df[(train_df["DIAG1"].isna()) & (train_df["PRDIAG1"] == 1)].shape[0]}')
print()

# Check the number of available dependent samples
# (when 'DIAG1' is not missing and 'PRDIAG1' is not 1)
print(f'Number of Available Dependent Samples: {train_df[(~train_df["DIAG1"].isna()) & (train_df["PRDIAG1"] != 1)].shape[0]}')

Missing Values in DIAG1: 1214
Missing Values in DIAG2: 53134
Missing Values in DIAG3: 79860

Number of Ruled Out Diagnoses in DIAG1: 940
Number of Ruled Out Diagnoses in DIAG2: 1054
Number of Ruled Out Diagnoses in DIAG3: 493

Number of Samples with Missing DIAG1 and PRDIAG1 equals to 1: 5

Number of Available Dependent Samples: 101337


### 4.1 - Load and the list of three-digit categories of ICD-9-CM

In [25]:
# Load the list of three-digit categories of ICD-9-CM
icd9cm_3dcat = pd.read_excel(os.path.join('..', 'data', 'raw', 'ICD9CM_3DCat.xlsx'), dtype=str)

icd9cm_3dcat.head()

,3D_CODE,DISEASE,CATEGORY_1,CATEGORY_2
0,001,Cholera,INFECTIOUS AND PARASITIC DISEASES,Intestinal infectious diseases
1,002,Typhoid and paratyphoid fevers,INFECTIOUS AND PARASITIC DISEASES,Intestinal infectious diseases
2,003,Other salmonella infections,INFECTIOUS AND PARASITIC DISEASES,Intestinal infectious diseases
3,004,Shigellosis,INFECTIOUS AND PARASITIC DISEASES,Intestinal infectious diseases
4,005,Other food poisoning (bacterial),INFECTIOUS AND PARASITIC DISEASES,Intestinal infectious diseases


### 4.2 - Employing the hierachical classifications of ICD-9-CM codes to prepare the target labels

In [26]:
# Map the three-digit categories of ICD-9-CM to 'DIAG1', 'DIAG2', and 'DIAG3',
# if 'PRDIAG1', 'PRDIAG2', and 'PRDIAG3' are not 1 respectively

def get_icd9cm_3dcat(diag, prdiag, category='CATEGORY_1'):
    try:
        if pd.notna(diag) and (pd.isna(prdiag) | prdiag != 1):
            if diag == 'V997-':
                return 'No diagnosis/disease or healthy'
            else:
                return icd9cm_3dcat[icd9cm_3dcat['3D_CODE'] == diag[:3]][category].values[0]
        else:
            return pd.NA
    except:
        print(f'Error: {diag}')
        print(f'Error: {prdiag}')

    
get_icd9cm_3dcat(train_df.iloc[0].DIAG1, train_df.iloc[0].PRDIAG1, category='CATEGORY_2')

'Certain traumatic complications and unspecified injuries'

In [27]:
# Map the three-digit categories of ICD-9-CM to 'DIAG1', 'DIAG2', and 'DIAG3',
# if 'PRDIAG1', 'PRDIAG2', and 'PRDIAG3' are not 1 respectively

y_train = train_df.apply(lambda x: get_icd9cm_3dcat(x.DIAG1, x.PRDIAG1, category='CATEGORY_1'), axis=1)
y_train_cat2 = train_df.apply(lambda x: get_icd9cm_3dcat(x.DIAG1, x.PRDIAG1, category='CATEGORY_2'), axis=1)

print(f'Dependent DataFrame with CATEGORY_1 Shape: {y_train.shape}')
print(f'Dependent DataFrame with CATEGORY_2 Shape: {y_train_cat2.shape}')

Dependent DataFrame with CATEGORY_1 Shape: (103486,)
Dependent DataFrame with CATEGORY_2 Shape: (103486,)


### 4.3 - Drop rows from both X_train, y_train with NA in y_train

In [28]:
print(f'Number of available dependent samples: {y_train.notna().sum()}')
print()

X_train = X_train[y_train.notna()]
y_train = y_train[y_train.notna()]

#X_train_cat2 = X_train[y_train_cat2.notna()]
y_train_cat2 = y_train_cat2[y_train_cat2.notna()]

print(f'Classification DataFrame Shape: {X_train.shape}')
print(f'Dependent DataFrame with CATEGORY_1 Shape: {y_train.shape}')
print(f'Dependent DataFrame with CATEGORY_2 Shape: {y_train_cat2.shape}')

Number of available dependent samples: 101337

Classification DataFrame Shape: (101337, 185)
Dependent DataFrame with CATEGORY_1 Shape: (101337,)
Dependent DataFrame with CATEGORY_2 Shape: (101337,)


## 5 - Classification models

### 5.1 - Model training

#### 5.1.1 - Logistic Regression - Baseline model

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

random_state = 42

##### 5.1.1.1 - Logistic Regression - Dropping the columns with missing values
Quantitative variables PASTVIS, HTIN, WTLB have missing values

In [30]:
# Train a logistic regression model with y_train
lr = LogisticRegression(random_state=random_state, max_iter=1000)
lr_cv_scores = cross_val_score(lr, X_train.iloc[:, 3:], y_train, cv=5, scoring='accuracy')

print(f'Logistic Regression CV Scores: {lr_cv_scores}')
print(f'Logistic Regression CV Mean Score: {np.mean(lr_cv_scores)}')

Logistic Regression CV Scores: [0.59739491 0.61293665 0.6078354  0.61079588 0.60378941]
Logistic Regression CV Mean Score: 0.6065504482009461


In [31]:
# Train a second logistic regression model with y_train_cat2
lr_cat2 = LogisticRegression(random_state=random_state, max_iter=1000)
lr_cat2_cv_scores = cross_val_score(lr_cat2, X_train.iloc[:, 3:], y_train_cat2, cv=5, scoring='accuracy')

print(f'Logistic Regression with CATEGORY_2 CV Scores: {lr_cat2_cv_scores}')
print(f'Logistic Regression with CATEGORY_2 CV Mean Score: {np.mean(lr_cat2_cv_scores)}')

/Users/sean7x/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Logistic Regression with CATEGORY_2 CV Scores: [0.4109927  0.41415038 0.41949968 0.42137465 0.41481226]
Logistic Regression with CATEGORY_2 CV Mean Score: 0.4161659333588327


##### 5.1.1.2 - Logistic Regression - Imputing the missing values - KNN Imputer
Quantitative variables PASTVIS, HTIN, WTLB have missing values

In [32]:
# Impute the missing quantitative values using KNN
from sklearn.impute import KNNImputer

n_neighbors = 5
imputer = KNNImputer(n_neighbors=n_neighbors)

quantitative_features_imputed_knn = imputer.fit_transform(X_train[quantitative_features])

In [33]:
# Concatenate the imputed quantitative features with the encoded categorical features
X_train_imputed_knn = pd.concat(
    [pd.DataFrame(quantitative_features_imputed_knn, columns=quantitative_features, index=X_train.index), X_train.iloc[:, 3:]],
    axis=1
)

In [34]:
# Train a logistic regression model with y_train and X_train_imputed_knn
lr_imputed_knn = LogisticRegression(random_state=random_state, max_iter=1000)
lr_imputed_knn_cv_scores = cross_val_score(lr_imputed_knn, X_train_imputed_knn, y_train, cv=5, scoring='accuracy')

print(f'Logistic Regression with Imputed Values CV Scores: {lr_imputed_knn_cv_scores}')
print(f'Logistic Regression with Imputed Values CV Mean Score: {np.mean(lr_imputed_knn_cv_scores)}')

Logistic Regression with Imputed Values CV Scores: [0.59892441 0.61303533 0.60911827 0.61301623 0.60674989]
Logistic Regression with Imputed Values CV Mean Score: 0.6081688265679233


##### 5.1.1.3 - Logistic Regression - Imputing the missing values - Maximum Likelihood Estimation
Quantitative variables PASTVIS, HTIN, WTLB have missing values

In [35]:
# Plot the boxplot for each of the quantitative features in X_train
chart(
    df=X_train[quantitative_features].melt(),
    x='variable:O',
    y='value:Q',
    title='Boxplot of Quantitative Features',
).mark_boxplot().interactive()

alt.Chart(...)

In [36]:
# Assume a normal distribution for 'HTIN' and 'WTLB'
# and a long-tailed distribution for 'PASTVIS',
# apply Maximum Likelihood Estimation (MLE) to impute the missing values

from scipy.stats import norm, expon
from scipy.optimize import minimize

def mle_normal(params, data):
    mu, sigma = params
    if sigma < 0:
        return np.inf
    return -np.sum(norm.logpdf(data, loc=mu, scale=sigma))

def mle_expon(params, data):
    lambda_ = params[0]
    if lambda_ < 0:
        return np.inf
    return -np.sum(expon.logpdf(data, scale=1/lambda_))

def impute_mle(data, distribution='normal'):
    if distribution == 'normal':
        mle = minimize(mle_normal, [data.mean(), data.std()], args=(data))
        return np.random.normal(mle.x[0], mle.x[1], data.isna().sum())
    elif distribution == 'exponential':
        mle = minimize(mle_expon, [1/data.mean()], args=(data))
        return np.random.exponential(1/mle.x[0], data.isna().sum())
    
def impute_mle(data, distribution='normal'):
    missing_values = data[data.isnull()]
    if distribution == 'normal':
        mle = minimize(mle_normal, [data.mean(), data.std()], args=(data.dropna()))
        imputed_values = np.random.normal(mle.x[0], mle.x[1], missing_values.shape[0])
    elif distribution == 'exponential':
        mle = minimize(mle_expon, [1/data.mean()], args=(data.dropna()))
        imputed_values = np.random.exponential(1/mle.x[0], missing_values.shape[0])
    return pd.Series(imputed_values, index=missing_values.index)


X_train_imputed_mle = X_train.copy()

# Impute the missing values in 'HTIN' and 'WTLB' using MLE
X_train_imputed_mle['HTIN'] = X_train_imputed_mle['HTIN'].fillna(impute_mle(X_train_imputed_mle['HTIN'], distribution='normal'))
X_train_imputed_mle['WTLB'] = X_train_imputed_mle['WTLB'].fillna(impute_mle(X_train_imputed_mle['WTLB'], distribution='normal'))

# Impute the missing values in 'PASTVIS' using MLE
X_train_imputed_mle['PASTVIS'] = X_train_imputed_mle['PASTVIS'].fillna(impute_mle(X_train_imputed_mle['PASTVIS'], distribution='exponential'))

# Check the result
X_train_imputed_mle[quantitative_features].isnull().sum()

/Users/sean7x/miniconda3/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


PASTVIS    0
HTIN       0
WTLB       0
dtype: int64

In [37]:
# # Train a logistic regression model with y_train and X_train_imputed_mle
lr_imputed_mle = LogisticRegression(random_state=random_state, max_iter=1000)
lr_imputed_mle_cv_scores = cross_val_score(lr_imputed_mle, X_train_imputed_mle, y_train, cv=5, scoring='accuracy')

print(f'Logistic Regression with MLE Imputed Values CV Scores: {lr_imputed_mle_cv_scores}')
print(f'Logistic Regression with MLE Imputed Values CV Mean Score: {np.mean(lr_imputed_mle_cv_scores)}')

Logistic Regression with MLE Imputed Values CV Scores: [0.59823367 0.61357805 0.61054917 0.61252282 0.60704594]
Logistic Regression with MLE Imputed Values CV Mean Score: 0.6083859297209956


In [ ]:
# Save the model
#model_path = os.path.join('..', 'models')
#model_name = type(lr).__name__

#joblib.dump(lr, os.path.join(model_path, f'{model_name}.joblib'))

#### 5.1.2 - Pipeline of other classification models

### 5.2 - Model evaluation

#### 5.2.1 - Metrics

In [39]:
# Predict with lr_imputed_mle
lr_imputed_mle.fit(X_train_imputed_mle, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [41]:
# Load the test dataset
test_df = pd.read_parquet(os.path.join(input_path, 'test.parquet'))
X_test = test_df.loc[:, features]

# Bin the AGE variable as age groups
X_test['AGE_GROUP'] = X_test['AGE'].apply(bin_age)

# Bin the REASON FOR VISIT classification summary of codes
# Map the corresponding `MODULE_2` to X_test
X_test['RFV1_MOD'] = X_test['RFV1'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
X_test['RFV2_MOD'] = X_test['RFV2'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)
X_test['RFV3_MOD'] = X_test['RFV3'].apply(lambda x: get_module(int(str(x)[:4])).strip() if pd.notna(x) else pd.NA)

# Bin the BMI as weight status
X_test['BMI_GROUP'] = X_test['BMI'].apply(bin_bmi)

# Bin the TEMPF as fever status
X_test['TEMPF_GROUP'] = X_test['TEMPF'].apply(bin_tempf)

# Bin the BPSYS as systolic blood pressure status
X_test['BPSYS_GROUP'] = X_test['BPSYS'].apply(bin_bpsys)

# Bin the BPDIAS as diastolic blood pressure status
X_test['BPDIAS_GROUP'] = X_test['BPDIAS'].apply(bin_bpdias)

# Normalize quantitative features
print(f'Normalize the Quantitative Features: {quantitative_features}')
X_test[quantitative_features] = scaler.transform(X_test[quantitative_features])

# Encode binary features
print(f'Encode the Binary Features: {binary_features}')
X_test[binary_features] = X_test[binary_features].map(lambda x: True if x == 1 else False)

# Encode non-binary categorical features
print(f'Encode the Non-binary Categorical Features: {non_binary_features}')
X_test = pd.get_dummies(X_test, columns=non_binary_features)

# Redefine the test DataFrame
X_test = X_test.loc[:, quantitative_features + binary_features + encoded_non_binary_features]

# Check the shape of the test DataFrame
print(f'Test DataFrame Shape: {X_test.shape}')

# Check the missing values in X_test
print(f'Missing Values in Test DataFrame: {X_test.isnull().sum()}')

# Impute the missing values in 'HTIN' and 'WTLB' using MLE
X_test['HTIN'] = X_test['HTIN'].fillna(impute_mle(X_test['HTIN'], distribution='normal'))
X_test['WTLB'] = X_test['WTLB'].fillna(impute_mle(X_test['WTLB'], distribution='normal'))

# Impute the missing values in 'PASTVIS' using MLE
X_test['PASTVIS'] = X_test['PASTVIS'].fillna(impute_mle(X_test['PASTVIS'], distribution='exponential'))

# Check the result
print(f'Missing Values in Test DataFrame: {X_test.isnull().sum()}')

# Predict with lr_imputed_mle
y_pred = lr_imputed_mle.predict(X_test)

Normalize the Quantitative Features: ['PASTVIS', 'HTIN', 'WTLB']
Encode the Binary Features: ['SEX', 'USETOBAC', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS']
Encode the Non-binary Categorical Features: ['INJDET', 'MAJOR', 'CASTAGE', 'AGE_GROUP', 'RFV1_MOD', 'RFV2_MOD', 'RFV3_MOD', 'BMI_GROUP', 'TEMPF_GROUP', 'BPSYS_GROUP', 'BPDIAS_GROUP']
Test DataFrame Shape: (34718, 185)
Missing Values in Test DataFrame: PASTVIS                                          6531
HTIN                                            17402
WTLB                                            11423
SEX                                                 0
USETOBAC                                            0
                                                ...  
BPSYS_GROUP_Prehypertension                         0
BPDIAS_GROUP_High diastolic blood pressure          0
BPDIAS_GROUP_Hypertension                           0
BPDIAS_GROUP_Low

/Users/sean7x/miniconda3/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/Users/sean7x/miniconda3/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


In [45]:
# Prepare the true labels
y_test = test_df.apply(lambda x: get_icd9cm_3dcat(x.DIAG1, x.PRDIAG1, category='CATEGORY_1'), axis=1)

# Check the shape of the true labels
print(f'True Labels Shape: {y_test.shape}')

True Labels Shape: (34718,)


In [51]:
# Check the predicted label of the first sample
print(f'Predicted Label of the First Sample: {y_pred[0]}')
print()

# Check the prediction probabilities of the first sample
y_pred_proba = lr_imputed_mle.predict_proba(X_test)
y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=lr_imputed_mle.classes_)
print(f'Prediction Probabilities of the First Sample:')
y_pred_proba_df.iloc[0].sort_values(ascending=False)

Predicted Label of the First Sample: DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS

Prediction Probabilities of the First Sample:


DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS                                                       0.525625
SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES    0.361491
CONGENITAL ANOMALIES                                                                                  0.019700
DISEASES OF THE RESPIRATORY SYSTEM                                                                    0.018886
SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS                                                           0.016000
ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS                                 0.015441
DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE                                                          0.008250
MENTAL DISORDERS                                                                                      0.008103
CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD                                                0.005473
I

In [50]:
# Check the true label of the first sample
print(f'True Label of the First Sample: {y_test.iloc[0]}')

True Label of the First Sample: DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS


In [55]:
# Check the prediction accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test[y_test.notna()], y_pred[y_test.notna()])
print(f'Prediction Accuracy: {accuracy}')

Prediction Accuracy: 0.5919882611885546


#### 5.2.2 - Examine cluster centroids

In [ ]:
# Check the cluster centers
cluster_centers = pd.DataFrame(clustering_model.cluster_centers_, columns=clustering_df_imputed.columns)

# Inverse transform the cluster centers
cluster_centers[quantitative_features] = scaler.inverse_transform(cluster_centers[quantitative_features])

# Check the result
cluster_centers

#### 5.2.3 - Visualization

## 6 - Extract text features from each cluster

#### 6.1 - Aggregate text data by group

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i < 5:
        print(k, v)

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i >= 5 and i < 10:
        print(k, v)

In [ ]:
for i, (k, v) in zip (range(len(variables)), variables.items()):
    if i >= 10:
        print(k, v)

In [ ]:
train_df.USETOBAC.unique()

In [ ]:
train_df[variables['visitReason']].head()

In [ ]:
train_df[variables['vitalSigns']].value_counts()

In [ ]:
train_df[variables['physicianDiagnoses']].head()

In [ ]:
print(train_df.PRDIAG1.unique())
print()

train_df.DIAG1[train_df.PRDIAG1.str.contains('not probable') | (train_df.PRDIAG1 == 'No')].value_counts()

In [ ]:
train_df[variables['presentSymptomsStatus']].value_counts()

In [ ]:
# Load custom function to combine text features
import sys
sys.path.append('../src/features/')

from combine_textual import combine_features


# Define the list of textual features to combine
textual_features = [
    'AGE', 'SEX', 'USETOBAC', 
    'MAJOR', 'RFV1', 'RFV2', 'RFV3', 
    'BMI', 'TEMPF', 'BPSYS', 'BPDIAS',
    'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'DMP',
    'DIAG1', 'DIAG2', 'DIAG3'
]

# Export the list of textual features
with open(os.path.join(file_path, 'textual_features.json'), 'w') as f:
    json.dump(textual_features, f)

# Combine the text features
train_df['CombinedText'] = train_df.apply(lambda x: combine_features(x, textual_features), axis=1)

train_df.CombinedText.head()

In [ ]:
train_df.CombinedText.notna().sum()

#### 6.2 - Preprocess text data

In [ ]:
import spacy
import re

In [ ]:
nlp = spacy.load('en_core_web_sm')

def preprocess_text(row):
    row = re.sub(r'(\d+),(\d+)', r'\1\2', row)
    row = re.sub(r'(\d+)-(\d+)', r'\1_\2', row)
    doc = nlp(row)
    processed_text = ' '.join(token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct)
    row = re.sub(r'(\d+)_(\d+)', r'\1-\2', row)
    return processed_text

In [ ]:
train_df['ProcessedText'] = train_df['CombinedText'].apply(preprocess_text)

In [ ]:
train_df.ProcessedText.head()

In [ ]:
# Save the preprocessed DataFrame
processed_file_path = os.path.join('..', 'data', 'processed')
train_df.to_csv(os.path.join(processed_file_path, f'train_{clustering_model_name}.csv'), index=False)

#### 6.3 - Calculate term frequencies

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Method 1
# Calculate the TF-IDF for each cluster,
# taking the ProcessedText of each cluster as the documents,
# and the ProcessedText of the entire dataset as the corpus

#clustered_text = train_df.groupby('cluster')['ProcessedText'].apply(lambda row: ' '.join(row)).reset_index()

#vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=5, max_df=0.7)
#tfidf_matrix = vectorizer.fit_transform(clustered_text['ProcessedText'])

#print(tfidf_matrix)

#tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
#tfidf_df

In [ ]:
# Method 2
# Calculate the TF-IDF of each row within each cluster
# Calculate the average TF-IDF for each cluster

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=5, max_df=0.7)
tfidf_matrix = vectorizer.fit_transform(train_df['ProcessedText'])

# Calculate the average TF-IDF for each cluster
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
cluster_tfidf = pd.concat([train_df['cluster'], tfidf_df], axis=1).groupby('cluster').mean()

cluster_tfidf

In [ ]:
# Punish the weight of '\d+_year_old' by multiplying it by 0.5, using the regex pattern

#tfidf_df = tfidf_df.apply(lambda row: row * 0.5 if re.match(r'\d+_year_old', row.name) else row)
#cluster_tfidf = cluster_tfidf.apply(lambda row: row * 0.5 if re.match(r'\d+_year_old', row.name) else row)

#cluster_tfidf

#### 6.4 - Generate word clouds for each group

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Plot the word cloud for each cluster basd on the average TF-IDF
for i in range(n_clusters):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(cluster_tfidf.loc[i])
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Cluster {i}')
    plt.show()

## 7 - Topic modeling

In [ ]:
import gensim
from gensim import corpora

random_seed = 42

In [ ]:
# Generate bigrams
def generate_bigrams(row):
    tokens = row.split()
    bigram_model = gensim.models.Phrases(tokens, min_count=5, threshold=100)
    tokens = [token for token in bigram_model[tokens]]
    return tokens


print(generate_bigrams(train_df['ProcessedText'].iloc[0]))
preprocessed_docs = train_df['ProcessedText'].apply(generate_bigrams)

In [ ]:
# Build dictionary and corpus
dictionary = corpora.Dictionary(preprocessed_docs)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]

In [ ]:
# Train LDA model
n_topics = 10

lda_model = gensim.models.LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=n_topics,
    random_state=random_seed,
    chunksize=100,
    passes=10,
)



### 7.1 - Visualize the topics

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis

pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

### 7.2 - Get the topic distribution for each cluster

In [ ]:
# Get the topic distribution for each document 
def get_avg_topic_distribution(cluster_data, topic_distribution, num_topics):
    cluster_topics = np.zeros((len(cluster_data.index), num_topics))

    # Update the distribution with the actual values
    for i, doc_index in enumerate(cluster_data.index.tolist()):
        for topic, prob in topic_distribution[doc_index]:
            cluster_topics[i, topic] = prob

    avg_topic_dist = np.mean(cluster_topics, axis=0)
    return avg_topic_dist


topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0.0)
clusters = train_df.groupby('cluster').ProcessedText

cluster_topics = []
for cluster, data in clusters:
    avg_topic_dist = get_avg_topic_distribution(data, topic_distribution, lda_model.num_topics)
    cluster_topics.append([avg_topic_dist.tolist()])

cluster_topics_df = pd.DataFrame(cluster_topics, columns=['avg_topic_distribution'])

cluster_topics_df

### 7.3 - Visualize the distribution of topics within each cluster

In [ ]:
import altair as alt

In [ ]:
# Reshape the dataframe to have a row for each cluster and topic
cluster_topics_heatmap_df = cluster_topics_df.avg_topic_distribution.apply(pd.Series).reset_index().rename(columns={'index': 'cluster'}).melt(id_vars='cluster', var_name='topic', value_name='probability')

heatmap = alt.Chart(cluster_topics_heatmap_df).mark_rect().encode(
    x='topic:O',
    y='cluster:O',
    color='probability:Q'
).properties(
    title='Average Topic Distribution for Each Cluster',
    width=400,
    height=400
)

heatmap